## DDIM with similar noise

In this notebook we will perform some experiments with Denoising Diffusion Implicit Models (DDIM). Unlike Denoising Diffusion Probabilistic Models (DDPM) DDIMs work deterministic, i.e. from one specific full noise picture it will always generate the same clear picture. Using a deterministic backwards process is essential here. If we used a probabilistic model, there would be a little noise added back into the picture after every backwards step, which would make the results differ heavily regardless of the starting point.

We will do 2 Experimernts:

1. Experiment (Patch condition): Change a small patch in the full noise picture and generate two clear images from it. You can choose the patch size and position yourself.
2. Experiment (Full noise condition): Change the full noise images in a way that the mathematical distance between the two full noise images used as a starting point is very small. You can choose how much the noise is changed by choosing a value of noise_scaling_factor yourself.

Each experiments will demonstrate different things:

1. Experiment: Changing a small patch in the full noise picture will not only change that exact patch in the clear picture, but the whole picture. This shows that diffusion models captures dependecies between pixels by capturing the probablity distribtutions in the training dataset.
2. Experiment: Changing the noise only a little only changes the resulting clear image a little.

As you will see there is always the euclidean distance and heatmap displayed to give you an objective and visual measure where and how the images are changed. Feel free to try out different values in  both experiments. Some interesting questions to research could be:

- Is there a correlation between the euclidean distance of the noise images and the euclidean distance of the clear images?
- Given a similar euclidean distance of noise images in both experiments, is the result changed more strongly in the patch condition or the full noise condition?
- In experiment 2: How much do you need to change the noise image to get substantially different results?




## Setup

In [ ]:
from functions import *

In [ ]:
import torch
import diffusers
from PIL import Image
from tqdm import tqdm
import os

In [ ]:
# setup
model_id = "google/ddpm-bedroom-256" # "google/ddpm-celebahq-256"
model = diffusers.UNet2DModel.from_pretrained(model_id)
ddpm_scheduler = diffusers.DDPMScheduler.from_pretrained(model_id)
ddpm_scheduler.set_timesteps(50)

In [ ]:
# input prepraration
image_size = model.config.sample_size # get image size
noise = torch.randn((1, 3, image_size, image_size)) # sample random noise

## 1. Experiment

In [ ]:
# setup
ddim_scheduler = diffusers.DDIMScheduler.from_pretrained(model_id)
ddim_scheduler.set_timesteps(50)

In [ ]:
# Letting the user choose patchsize and position
patch_size = None 
patch_position_x = None
patch_position_y = None
while patch_position_x == None or patch_position_x < 0 or patch_position_x > 256:
    try:
        patch_position_x = int(input("Please choose a x position for the patch between 0 and 256"))
    except ValueError:
        print("Please insert an integer.")
while patch_position_y == None or patch_position_y < 0 or patch_position_y > 256:
    try:
        patch_position_y = int(input("Please choose a y position for the patch between 0 and 256"))
    except ValueError:
        print("Please insert an integer.")
while patch_size == None or patch_position_x + patch_size > 256 or patch_position_y + patch_size > 256:
    try:
        patch_size = int(input("Please choose a patch size."))
        if patch_position_x + patch_size > 256 or patch_position_y + patch_size > 256:
            print("Patch size is to big. Please choose a smaller one.")
    except ValueError:
        print("Please insert an integer.")


# prepare input
noise = torch.randn((1, 3, image_size, image_size)) # sample random noise
noises = [noise.clone() for _ in range(2)] # duplicate noise
noises[1][:,:,patch_position_y:patch_position_y+patch_size,patch_position_x:patch_position_x+patch_size] = torch.randn((1, 3, patch_size, patch_size)) # change a small patch in one of the full noise pictures
euclidean = torch.norm((noises[0]-noises[1])).item() # calculate euclidean distance
show_table([[tensor_as_html(noises[0]), tensor_as_html(noises[1]), tensor_as_html(torch.abs(noises[0]-noises[1]).mean(dim=1,keepdim=True).repeat(1,3,1,1))], ["", f"Euclidean distance: {euclidean}", "Heatmap"]])#display images, euclidean, heatmap

In [ ]:
# display the images alternately
from time import sleep
for i in range(10):
    show_images(noises[i % 2])
    sleep(0.5)

In [ ]:
# output generation
images = list()
for current in noises:
    for t in tqdm(ddim_scheduler.timesteps):
        with torch.no_grad():
            predicted_noise = model(current, t).sample
            current = ddim_scheduler.step(predicted_noise, t, current).prev_sample
    images.append(current)

In [ ]:
# show output
euclidean = torch.norm((images[0]-images[1])).item() # calculate euclidean distance
show_table([[tensor_as_html(images[0]), tensor_as_html(images[1]), tensor_as_html(torch.abs(images[0]-images[1]).mean(dim=1,keepdim=True).repeat(1,3,1,1))], ["", f"Euclidean distance: {euclidean}", "Heatmap"]])#display images, euclidean, heatmap

In [ ]:
# save input and output
for i in range(len(noises)):
    tensor_as_image(noises[i]).save(f"../output/similar_ddim_noise_{i}.png")
    tensor_as_image(images[i]).save(f"../output/similar_ddim_image_{i}.png")

## 2. Experiment

In [ ]:
#Letting the user choose a scaling factor
noise_scaling_factor = None
while noise_scaling_factor == None or noise_scaling_factor < 0.0 or noise_scaling_factor > 1.0:
    try:
        noise_scaling_factor = float(input("Please choose a noise scaling factor between 0.0 and 1.0"))
        if noise_scaling_factor < 0.0 or noise_scaling_factor > 1.0:
            print("Please choose a value between 0.0 and 1.0")
    except ValueError:
        print("Please insert a float.")
        


# prepare input
noise = torch.randn((1, 3, image_size, image_size)) # sample random noise
noises = [noise.clone() for _ in range(2)] # duplicate noise
noises[1] = ((1-noise_scaling_factor**2)**0.5) * noises[1] + noise_scaling_factor * torch.randn((1, 3, image_size, image_size)) # change one of the full noise pictures by adding newly generated noise scaled down heavily
euclidean = torch.norm((noises[0]-noises[1])).item() # calculate euclidean distance
show_table([[tensor_as_html(noises[0]), tensor_as_html(noises[1]), tensor_as_html(torch.abs(noises[0]-noises[1]).mean(dim=1,keepdim=True).repeat(1,3,1,1))], ["", f"Euclidean distance: {euclidean}", "Heatmap"]])#display images, euclidean, heatmap

In [ ]:
# output generation
images = list()
for current in noises:
    for t in tqdm(ddim_scheduler.timesteps):
        with torch.no_grad():
            predicted_noise = model(current, t).sample
            current = ddim_scheduler.step(predicted_noise, t, current).prev_sample
    images.append(current)

In [ ]:
# show output
euclidean = torch.norm((images[0]-images[1])).item() # calculate euclidean distance
show_table([[tensor_as_html(images[0]), tensor_as_html(images[1]), tensor_as_html(torch.abs(images[0]-images[1]).mean(dim=1,keepdim=True).repeat(1,3,1,1))], ["", f"Euclidean distance: {euclidean}", "Heatmap"]])#display images, euclidean, heatmap